In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import os
import collections
import math 
from sklearn import preprocessing
from IPython.display import display, HTML
from experiments.models import Analysis
from compound.models import Pathway, CompoundPathway, Compound
from data.models import Peak, Dataset, PeakDTSample, PeakQCSample
from fileupload.models import Sample
from collections import defaultdict
from experiments.pipelines.pipeline_rpy2 import Rpy2PipelineMetadata
from experiments.pipelines.helpers import convert_to_dataframe

### Sampling the p-values over a pathways with a number of different peaks ( none random )

In [11]:
no_change_df = calc_av_p_scores(100)
display (no_change_df)

,t-stat,p-value
0,,
eighty,-6.238990,0.000835
forty,-6.178943,0.000914
twenty,-6.041539,0.001002
ten,-5.959876,0.001285
six,-5.907006,0.001740
four,-5.868487,0.001930
two,-5.584648,0.003279


### Adding a pecentage of random peaks to the pathways STD = 5

In [33]:
random_50_df = calc_av_p_scores(100,50)
display (random_50_df)

,t-stat,p-value
0,,
eighty,-6.188002,0.000874
forty,-6.183936,0.000905
twenty,-5.951279,0.001129
ten,-5.864376,0.001474
six,-5.825705,0.002362
four,-5.463280,0.002924
two,-4.910249,0.010971


In [34]:
random_100_df = calc_av_p_scores(100,100)
display (random_100_df)

,t-stat,p-value
0,,
eighty,-6.150526,0.000907
forty,-6.131710,0.000997
twenty,-6.028928,0.001290
ten,-5.858432,0.002011
six,-5.879864,0.002207
four,-5.513818,0.003230
two,-4.542326,0.013898


In [38]:
random_200_df = calc_av_p_scores(100,200)
display (random_200_df)

,t-stat,p-value
0,,
eighty,-6.036844,0.000971
forty,-6.138930,0.001076
twenty,-5.958228,0.001525
ten,-5.691877,0.002579
six,-5.218225,0.004084
four,-4.927801,0.006884
two,-2.562668,0.062462


### Adding a pecentage of random peaks to the pathways STD = 1

In [40]:
random_50_df_1 = calc_av_p_scores(100,50)
display (random_50_df_1)

,t-stat,p-value
0,,
eighty,-6.109979,0.000837
forty,-6.145938,0.000904
twenty,-5.841589,0.001244
ten,-5.772518,0.001475
six,-5.785008,0.002047
four,-5.643737,0.003086
two,-5.029119,0.007907


In [30]:
random_100_df_1 = calc_av_p_scores(100,100)
display (random_100_df_1)

,t-stat,p-value
0,,
eighty,-6.160683,0.000917
forty,-6.197592,0.000957
twenty,-6.126346,0.001091
ten,-5.933496,0.001607
six,-5.626259,0.002597
four,-5.486142,0.004737
two,-4.300323,0.018063


In [37]:
random_200_df_1 = calc_av_p_scores(100,200)
display (random_200_df_1)

,t-stat,p-value
0,,
eighty,-6.206872,0.000902
forty,-6.051215,0.001055
twenty,-6.066777,0.001279
ten,-5.813536,0.001905
six,-5.199970,0.004160
four,-4.816737,0.009429
two,-3.045534,0.040065


### Nothing is changing, it's all random

In [39]:
all_random = calc_av_p_scores(100, random = True)
display (all_random)

,t-stat,p-value
0,,
twenty,0.262863,0.460380
six,-0.222342,0.491342
ten,0.259963,0.493589
four,-0.089219,0.497327
forty,-0.025718,0.515756
eighty,0.075168,0.517361
two,0.189110,0.544673


### Constuct the df*num_iterations and calculate the mean t-test scores


In [3]:
def calc_av_p_scores(num_iterations, percent=None, random=False):
    
    for it in range(num_iterations):
        fc_df = construct_fc_df(random) #Construct the fold change DF
        pathway_acts = get_pathway_act(fc_df)

        #If we want to add a percentatge of random peaks
        if percent is not None:
            add_random_df = add_random_peaks(fc_df, percent)
            pathway_acts = get_pathway_act(add_random_df)
        #Calulate the t-test results, if it is the first iteration, initialise the df.
        if it == 0:   
            t_results = get_t_test(pathway_acts).sort_index(axis=0)      
        
        else:  
            t_results = t_results + get_t_test(pathway_acts).sort_index(axis=0)      

    final = t_results/num_iterations
    new_df = final.sort_values(by='p-value', ascending = True)
    
    return new_df


### Construct a severeal pathways with 2 fold changes in the data (noise with std of 5 added)

In [10]:
sample_fnames = [u'Control_4.mzXML', u'Control_3.mzXML', u'Control_2.mzXML', 
           u'Control_1.mzXML', u'2_fold_1.mzXML', u'2_fold_2.mzXML', u'2_fold_3.mzXML', u'2_fold_4.mzXML']
data_names ={"two":2, "four":4, "six":6, "ten":10, "twenty":20, "forty":40, "eighty":80}

In [4]:
def construct_fc_df(random=False):
    sample_fnames = [u'Control_4.mzXML', u'Control_3.mzXML', u'Control_2.mzXML', 
           u'Control_1.mzXML', u'2_fold_1.mzXML', u'2_fold_2.mzXML', u'2_fold_3.mzXML', u'2_fold_4.mzXML']
    data_names ={"two":2, "four":4, "six":6, "ten":10, "twenty":20, "forty":40, "eighty":80}

    if not random:
        data = [12.0, 6.0, 12.0, 6.0, -17.0, -7.0, -7.0, -17.0]
    else:
        data = data = np.random.normal(0, 1, 8)
    pk_samp_intensities = []
    for name, num in data_names.iteritems():
        for n in range(num):
                peak_int_list =[]
                peak_int_list.append(name) 
                data_noise = data + np.random.normal(0, 5, len(data))
                peak_int_list.extend(list(data_noise)) #The intensities of all the samples for this peak.
                pk_samp_intensities.append(peak_int_list)

    fc_df = pd.DataFrame(pk_samp_intensities).set_index([0])

    fc_df.columns=sample_fnames
    fc_df.index.name = "ms1_peak_id"
    fc_df.columns.name ="sample_name"

    scaled_data = preprocessing.scale(np.array(fc_df), axis=1)

    fc_df[sample_fnames] = scaled_data
    
    
    return fc_df



### Set up a DF calcutating the T-test scores and the number of metabolites

In [5]:
from scipy.stats import ttest_ind
condition_1 = [u'Control_4.mzXML', u'Control_3.mzXML', u'Control_2.mzXML', u'Control_1.mzXML']
condition_2 = [u'2_fold_1.mzXML', u'2_fold_2.mzXML', u'2_fold_3.mzXML', u'2_fold_4.mzXML']


### Method to return p-value and t-test for an activity DF

In [6]:
def get_t_test(noisy_df):

    t_test_list = []

    for pathway, row in noisy_df.iterrows():
        c1 = noisy_df.loc[pathway, condition_1].values
        c2 = noisy_df.loc[pathway, condition_2].values
        path_params = [pathway] + list(ttest_ind(c1,c2))
        t_test_list.append(path_params)

    t_test_df = pd.DataFrame(t_test_list).set_index([0])
    t_test_df.columns = ['t-stat', 'p-value']

    t_test_df = t_test_df.sort_values(by='p-value', ascending = True)

    return t_test_df


### A method to add a percentage of random peaks to a peak intensity df

In [7]:
def add_random_peaks(int_df, percent):

    #For each of the pathways add random peaks
    rand_peak_list = []   
    for name in data_names:

        df_path = int_df.ix[name]
        num_peaks = math.ceil((df_path.shape[0])*(percent/100.0))

        num_samples = df_path.shape[1]
        
        #Generate random peaks for that pathway
        
        for p in range(int(num_peaks)):
            rand_peaks = []
            data = np.random.normal(0, 5, num_samples)
            rand_peaks.append(name)
            rand_peaks.extend(list(data))
            rand_peak_list.append(rand_peaks)

    #Construct a DF for the new peak list
    ran_df = pd.DataFrame(rand_peak_list).set_index([0])
    ran_df.columns=sample_fnames
    
    #Add the random peaks to the original DF
    new_df = pd.concat([int_df, ran_df])
    
    scaled_data = preprocessing.scale(np.array(new_df), axis=1)
    new_df[sample_fnames] = scaled_data

    
    return new_df
    
   

### Method to return the peak activities given a peak DF composed of pathways

In [8]:
def get_pathway_act(int_df):
    #For all of the pathways get all of the peak IDs
    pathway_activities = []
    for name in data_names:
        pathway_peaks = int_df.ix[name] #DF selected from peak IDs.
        w, d, c = np.linalg.svd(np.array(pathway_peaks))
        pw_act_list = []
        pw_act_list.append(name)
        pw_act_list.extend(list(c[0]))

        pathway_activities.append(pw_act_list)
        
        
        activity_df = pd.DataFrame(pathway_activities).set_index([0])
        activity_df.columns=int_df.columns
        activity_df.index.name = "Pathways"


    return activity_df
